# Image Classification of Documents

## 1. Setup
To prepare your environment, you need to install some packages and enter credentials for the Watson services.

### 1.1 Install the necessary packages
You need the latest versions of these packages:
python-swiftclient: is a python client for the Swift API.

### Install IBM Cloud Object Storage Client: 

In [ ]:
!pip install ibm-cos-sdk

### Now restart the kernel by choosing Kernel > Restart.

### 1.2 Import packages and libraries
Import the packages and libraries that you'll use:

In [1]:
import keras
import PIL
import numpy as np



from PIL import Image

from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import Adam

import ibm_boto3
from botocore.client import Config
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils

from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


## 2. Configuration
Add configurable items of the notebook below

### 2.1 Add your service credentials for Object Storage
You must create Object Storage service on IBM Cloud. To access data in a file in Object Storage, you need the Object Storage authentication credentials. Insert the Object Storage authentication credentials as credentials_1 in the following cell after removing the current contents in the cell.

In [3]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IBM_API_KEY_ID': '5j3Ot-9_c4Q4qwAAG9IehVYKDg-ovhJ38F95g1frNYGa',
    'IAM_SERVICE_ID': 'iam-ServiceId-917d87a5-7c76-4e5e-832f-1871587eaa77',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'try1e5849cebfc54c92a72ad6fcde9a73af',
    'FILE': 'KnowledgeBase_CodePattern_sample1.docx'
}


### 2.2 Global Variables
Add global variables.

In [5]:
STANDARD_SIZE=(224,224) # Size of all Input Images
num_classes=5 # Number of Classes the model needs to classify
filenames=["cheque_4.jpg","pancard_17.jpg","passport_28.jpg","drivinglicense_26.jpg","passport_27.jpg","none_16.jpg","pancard_16.jpg","cheque_5.jpg","none_15.jpg","drivinglicense_27.jpg"] # Names of the Input files present in Object Storage

## 3. Persistence and Storage

### 3.1 Configure Object Storage Client

In [4]:
# Ensure you name the variable containing Object Storage Credentials as credentials_1
cos = ibm_boto3.client('s3',
                    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
                    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
                    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
                    config=Config(signature_version='oauth'),
                    endpoint_url=credentials_1['ENDPOINT'])

def get_file(filename):
    '''Retrieve file from Cloud Object Storage'''
    fileobject = cos.get_object(Bucket=credentials_1['BUCKET'], Key=filename)['Body']
    return fileobject

## 4. Classification

### 4.1 Dataset Creation

In [2]:
'''Converting Data Format according to the backend used by Keras
'''
datagen=keras.preprocessing.image.ImageDataGenerator(data_format=K.image_data_format())

In [12]:
def convert_to_image(X_train):
    '''Function to convert all Input Images to the STANDARD_SIZE and create Training Dataset
    '''
    for f in filenames:
        fobj=get_file(f)
        #print(type(fobj))
        img= PIL.Image.open(fobj)
        img = img.resize(STANDARD_SIZE)
        img=np.array(img)
        X_train.append(img)
        #print(X_train)
    #print(len(X_train))
    return X_train

def convert_to_label(y_train):
    '''Function to Append Labels to Training Data
    '''
    y=[]
    for f in filenames:
        y.append(f.split("_")[0])
    labelencoder_y = LabelEncoder()
    y_train_temp= labelencoder_y.fit_transform(y)
    #print(y_train_temp)
    y_train_temp=np_utils.to_categorical(y_train_temp, num_classes)
    #print(type(y_train_temp))
    return y_train_temp
#convert_to_label(y_train)

### 4.2 Build the Model

In [8]:
vgg16_model = keras.applications.vgg16.VGG16() # Keras Functional API
print("### VGG16 Model ###")
vgg16_model.summary()
print("###################################################################")
print("\n")
model = Sequential() #Iterate over the functional layers and add it as a stack
for layer in vgg16_model.layers:
    model.add(layer)
model.layers.pop() # We don't want 1000 classes, so remove the last layer

for layer in model.layers: #Since the model is already trained with certain weights, we dont want to change it. Let it be the same
    layer.trainable = False

model.add(Dense(num_classes, activation='sigmoid')) # Add the last layer

print("### Tweaked VGG16 Model ###")
model.summary()
print("###################################################################")

### VGG16 Model ###
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0

In [9]:
# Compile the Model built above
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
'''
Prepare Training Data to be fed into the Model
'''
X_train= []
y_train= []
X_train=np.array(convert_to_image(X_train)) # ndarray of Training Data
y_train=np.array(convert_to_label(y_train))

In [14]:
datagen.fit(X_train)

In [ ]:
'''
Train your model
'''
model.fit_generator(datagen.flow(X_train, y_train, batch_size=5),
                    steps_per_epoch=5, epochs=5)


Epoch 1/5
5/5 [==============================] - 94s 19s/step - loss: 1.6069 - acc: 0.1200
Epoch 2/5
1/5 [=====>........................] - ETA: 1:15 - loss: 1.6015 - acc: 0.6000

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(y_train)
#y_train = np_utils.to_categorical(y_train, num_classes)
#y_test = np_utils.to_categorical(y_test, num_classes)
type(x_train)